This will serve as the notebook for testing my topic modeling portion of this project.

After verifying that everything is working, it will be changed to a .py file to be runnable on new incoming data

In [36]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re

# from topicModelingFunctions import text_preproc

import spacy
from spacy.tokens import DocBin
from nltk.corpus import stopwords
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [38]:
df = pd.read_csv(r"data/temp/2014_funny_cComments.csv", usecols=['score', 'body', 'flag', 'cComment'])
df.head()

,score,body,flag,cComment
0,1,Ain't no half steppin,okay,aint half steppin
1,7,Inside the gas tank cover or on top of a tire,okay,inside gas tank cover top tire
2,12,Brought to you by /r/SummerReddit,okay,brought rsummerreddit
3,1,This makes me want to murder.,okay,makes want murder
4,3,I know someone who got a DUI sleeping it off i...,okay,know someone got dui sleeping car saying cops ...


## Try Spacy Text Modeling

In [49]:
df = df[~df.cComment.isna()]

In [50]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
stop_words = stopwords.words('english')

In [51]:
# keys = np.unique(df.flag)
# values = class_weight.compute_class_weight(class_weight='balanced', classes=keys, y=df.flag)
# class_weights = dict(zip(keys, values))
# class_weights

In [52]:
X_train, X_test, y_train, y_test = train_test_split(df.cComment, df.flag, test_size=0.15)  
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)  # Create training, testing, and validation splits


In [53]:
# Check %ages of splits
print(len(X_train)/len(df))
print(len(X_test)/len(df))
print(len(X_val)/len(df))

0.7224999054063929
0.15000004332531622
0.12750005126829086


In [64]:
zipVal = zip(X_val, y_val)
zipTrain = zip(X_train, y_train)
zipTest = zip(X_test, y_test)

In [65]:
def make_docs(zippedList, outfile):
    """
    Function to transform our data into docs for spacy

    Args:
        data (zip): zipped list of sentences and categories to transform
        outfile (str): path to save the model to
    """
    db = DocBin()
    for comment, label in zippedList:
        # print(comment, label)
        doc = nlp.make_doc(comment)
        if label == 'okay':
            doc.cats['okay'] = 1
            doc.cats['bad'] = 0
            doc.cats['good'] = 0
            
        elif label == 'bad':
            doc.cats['okay'] = 0
            doc.cats['bad'] = 1
            doc.cats['good'] = 0
            
        else:
            doc.cats['okay'] = 0
            doc.cats['bad'] = 0
            doc.cats['good'] = 1
        
        db.add(doc)
    db.to_disk(outfile)
            

In [66]:
print("Starting")
make_docs(zipVal, r"data/models/spacy/valid.spacy")
print("Finished Val")
make_docs(zipTrain, r"data/models/spacy/train.spacy")
print("Finished Train")
make_docs(zipTest, r"data/models/spacy/test.spacy")
print("Finished Test")

Starting
Finished Val


MemoryError: Unable to allocate 7.52 GiB for an array with shape (77645270, 13) and data type uint64

In [58]:
config = {
    "threshold": 0.5,
    "model": DEFAULT_MULTI_TEXTCAT_MODEL
}

nlp.add_pipe("textcat_multilabel", config=config)

## Start BERT modeling

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)  # tfhub_handle_preprocess from the "Choose a BERT model to fine-tune" code at https://www.tensorflow.org/text/tutorials/classify_text_with_bert
bert_model = hub.KerasLayer(tfhub_handle_encoder)  # tfhub_handle_encoder from the "Choose a BERT model to fine-tune" code at https://www.tensorflow.org/text/tutorials/classify_text_with_bert

def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='body')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

classifier_model =build_Classifier_model()
tf.keras.utils.plot_model(classifier_model)  # Check model structure


In [ ]:
#### Model Training

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Multi-class classification
metrics = tf.metrics.Accuracy()

#### Optimizer

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()  ##### Must change to our actual training data instead of train_ds
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)  # first 10% of training steps to warmup optimizer

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

#### Load BERT model and train

In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
print(f"Training model with {tfhub_handle_encoder}")
history = classifier_model.fit(x=train_ds,  #### Must change to actual training data
                               validation_data=val_ds,   #### Must change to actual validation data
                               epochs=epochs)

#### Evaluate Model

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)   #### Must change to actual test data
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

#### Export Model

In [ ]:
dataset_name = "redditPredict"
saved_model_path = './data/models/{}_bert'.format(dataset_name.replace('/', '_'))
classifier_model.save(saved_model_path, include_optimizer=False)

#### Fine Tuning BERT